In [4]:
# Importação de Bibliotecas
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
# Carregar e Preparar os Dados
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df = pd.read_csv("DATA.csv")

In [6]:
# Limpeza dos Dados
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)


In [7]:
# Mapeie os valores 1 para 'M' e 2 para 'F'
df['sex_of_casualty'] = df['sex_of_casualty'].map({1: 'M', 2: 'F'})


In [11]:
# Filtrar valores inválidos em 'sex_of_casualty' e 'age_band_of_casualty'
df_filtered = df[(df['sex_of_casualty'] != -1) & (df['sex_of_casualty'] != 9) & (df['age_band_of_casualty'] != -1)]
df_sampled = df_filtered

>   # Pergunta Orientada a Dados: "Quais são os padrões e tendências na gravidade dos acidentes de trânsito com base no perfil demográfico das vítimas (idade e sexo)?"


In [13]:
# Análise e Visualização da Gravidade dos Acidentes por Sexo
severity_sex_contingency = pd.crosstab(df_filtered['casualty_severity'], df_filtered['sex_of_casualty'])
severity_sex_rate = severity_sex_contingency.apply(lambda r: r/r.sum()*100, axis=1)
print("Contagem de acidentes por gravidade e sexo:")
print(severity_sex_contingency)
print("\nFrequência relativa de acidentes por gravidade e sexo:")
print(severity_sex_rate)

Contagem de acidentes por gravidade e sexo:
sex_of_casualty        F      M
casualty_severity              
1                    195    592
2                   3748   7877
3                  19072  28342

Frequência relativa de acidentes por gravidade e sexo:
sex_of_casualty            F          M
casualty_severity                      
1                  24.777637  75.222363
2                  32.240860  67.759140
3                  40.224406  59.775594


>   alt.Chart(df_filtered).mark_bar(): Cria um gráfico de barras usando o DataFrame filtrado.
>   encode: Define os eixos x (sexo das vítimas) e y (contagem de acidentes) e a cor das barras (gravidade dos acidentes).
>   properties: Define o título do gráfico.
>   interactive(): Adiciona interatividade ao gráfico.
>   display(): Exibe o gráfico.

In [ ]:
# Habilita o VegaFusion Data Transformer
alt.data_transformers.enable('vegafusion')

# ... (seu código para criar df_sampled OU df_agg, escolha um dos dois)

# Cria o gráfico com os dados preparados (df_sampled ou df_agg)
bar_chart_severity_sex = alt.Chart(df_sampled).mark_bar().encode(
    x=alt.X('sex_of_casualty:N', axis=alt.Axis(labelAngle=-45), title='Sexo'),
    y=alt.Y('count()', title='Contagem'),  # ou 'count:Q' se usar df_agg
    color=alt.Color('casualty_severity:N', title='Gravidade'),
    tooltip=['sex_of_casualty', 'casualty_severity', 'count()']  # ou 'count' se usar df_agg
).properties(
    title='Gravidade dos Acidentes por Sexo'
).interactive()

bar_chart_severity_sex.display()

>    Similar à análise por sexo, mas agora agrupando por faixa etária (age_band_of_casualty).

In [ ]:
# Análise e Visualização da Gravidade dos Acidentes por Faixa Etária
severity_age_contingency = pd.crosstab(df_filtered['casualty_severity'], df_filtered['age_band_of_casualty'])
severity_age_rate = severity_age_contingency.apply(lambda r: r/r.sum()*100, axis=1)
print("Contagem de acidentes por gravidade e faixa etária:")
print(severity_age_contingency)
print("\nFrequência relativa de acidentes por gravidade e faixa etária:")
print(severity_age_rate)

>   Similar ao gráfico de barras por sexo, mas agora agrupando por faixa etária (age_band_of_casualty).

In [ ]:

# Gráfico de barras empilhadas para gravidade por faixa etária
bar_chart_severity_age = alt.Chart(df_filtered).mark_bar().encode(
    x=alt.X('age_band_of_casualty:N', axis=alt.Axis(labelAngle=-45), title='Faixa Etária'),
    y=alt.Y('count()', title='Contagem'),
    color=alt.Color('casualty_severity:N', title='Gravidade'),
    tooltip=['age_band_of_casualty', 'casualty_severity', 'count()']
).properties(
    title='Gravidade dos Acidentes por Faixa Etária'
).interactive()

bar_chart_severity_age.display()

>   groupby('casualty_severity')['age_of_casualty'].agg(['mean', 'std']): Agrupa os dados por gravidade (casualty_severity) e calcula a média e o desvio padrão da idade das vítimas (age_of_casualty).
>   reset_index(): Reseta o índice do DataFrame para torná-lo mais legível.
>   print: Exibe as estatísticas calculadas

In [ ]:
# Medidas Estatísticas por Gravidade
# Média e Desvio Padrão da Idade das Vítimas para Cada Nível de Gravidade
severity_age_stats = df_filtered.groupby('casualty_severity')['age_of_casualty'].agg(['mean', 'std']).reset_index()
print("\nMédia e Desvio Padrão da Idade das Vítimas por Gravidade:")
print(severity_age_stats)


>   plt.figure(figsize=(12, 6)): Cria uma figura com tamanho especificado.
>   sns.barplot: Cria um gráfico de barras mostrando a média da idade das vítimas por gravidade.
>   plt.errorbar: Adiciona barras de erro ao gráfico para representar o desvio padrão.
>   plt.title, plt.xlabel, plt.ylabel: Define o título do gráfico e os rótulos dos eixos.
>   plt.show(): Exibe o gráfico.

In [ ]:
# Visualização da Média e Desvio Padrão da Idade das Vítimas por Gravidade
plt.figure(figsize=(12, 6))
sns.barplot(x='casualty_severity', y='mean', data=severity_age_stats)
plt.errorbar(x=range(len(severity_age_stats)), y=severity_age_stats['mean'], yerr=severity_age_stats['std'], fmt='none', c='black', capsize=5)
plt.title('Média e Desvio Padrão da Idade das Vítimas por Gravidade')
plt.xlabel('Gravidade')
plt.ylabel('Idade Média das Vítimas')
plt.show()